In [1]:
import pandas as pd
import seaborn as sns
import os
from sklearn.compose import make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
print(os.getcwd())

/Users/matheus/Projects/steam_analysis/steam-analysis/notebooks/analysis


In [4]:
os.chdir("../..")

In [5]:
from notebooks.analysis.tranfs import (
    LowerCaseColumnsTransformer,
    BinaryColumnsTransformer,
    FillNATransformers,
    DropNATransformers,
    CounterColumnsTransformer,
    DateColumnsTransformer,
    ArithmeticColumnTransformer,
    DropColumnsTransformer,
    DataframeFilterTransformer,
    CreateTargetColumns,
)

In [6]:
steam_df = pd.read_csv("data/raw/games.csv")

In [9]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    steps=[
        ("lowercase_columns", LowerCaseColumnsTransformer()),
        (
            "create_binary_columns",
            BinaryColumnsTransformer(
                columns=["publishers", "support_email", "support_url"]
            ),
        ),
        ("fill_NA_columns", FillNATransformers(columns=["tags", "about_the_game"])),
        (
            "drop_na_columns",
            DropNATransformers(columns=["screenshots", "movies", "genres"]),
        ),
        (
            "create_counter_columns",
            CounterColumnsTransformer(
                ["tags", "screenshots", "movies", "supported_languages"]
            ),
        ),
        ("create_date_columns", DateColumnsTransformer("release_date")),
        (
            "create_total_reviews_columns",
            ArithmeticColumnTransformer("positive", "negative", "add", "total_reviews"),
        ),
        (
            "create_perceived_quality_columns",
            ArithmeticColumnTransformer(
                "positive", "negative", "truediv", "perceived_quality"
            ),
        ),
        ("drop_columns", DropColumnsTransformer("appid")),
        (
            "filter_dataframe",
            DataframeFilterTransformer(
                filter_dict={
                    "metacritic_score": 0,
                    "total_reviews": 10,
                    "year": 2023,
                    "genres": "Indie",
                }
            ),
        ),
        "create_target_columns",
        CreateTargetColumns(threshold=500),
    ]
)

In [10]:
pipeline.fit_transform(steam_df)

,name,release_date,estimated_owners,peak_ccu,required_age,price,dlc_count,about_the_game,supported_languages,full_audio_languages,reviews,header_image,website,support_url,support_email,windows,mac,linux,metacritic_score,metacritic_url,user_score,positive,negative,score_rank,achievements,recommendations,notes,average_playtime_forever,average_playtime_two_weeks,median_playtime_forever,median_playtime_two_weeks,developers,publishers,categories,genres,tags,screenshots,movies,has_publishers,has_support_email,has_support_url,n_tags,n_screenshots,n_movies,n_supported_languages,month,day,year,total_reviews,perceived_quality
68049,Under the Sky World,2023-01-04,0 - 20000,14,0,14.39,0,'Under the Sky World' is an orthodox casual RP...,"'Traditional Chinese', 'English', 'Japanese', ...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,bokibokiofficial@gmail.com,True,False,False,0,NaN,0,10,0,NaN,0,0,NaN,0,0,0,0,サークル☆フェアリーフラワー,BokiBoki Games,Single-player,"Adventure,Casual,Indie,RPG","JRPG,Sexual Content,Casual,Hentai,Female Prota...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,True,True,False,17,9,2,4,1,4,2023,11,10.000000
68069,NoEL - Nothing on Elysion Line,2023-01-05,0 - 20000,6,0,17.49,0,"게임 기본 정보 =============== ◎ 전투를 통해 덱을 수정하고, 장비를...",'Korean',['Korean'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,xenogames00@gmail.com,True,False,False,0,NaN,0,7,3,NaN,0,0,NaN,0,0,0,0,Xenogames,Xenogames,Single-player,"Adventure,Casual,Indie,RPG,Strategy","Card Game,Rogue-like,Anime,Visual Novel,Roguel...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,True,True,False,20,8,1,1,1,5,2023,11,1.750000
68074,ILLUMINATI,2023-01-06,0 - 20000,7,0,21.24,0,Battle for world domination in this darkly sat...,'English',['English'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://illuminati.game,https://illuminati.game,support@radiofree.net,True,True,False,0,NaN,0,11,5,NaN,0,0,"Some depictions of bloody violence, mortal per...",0,0,0,0,"Derek Pearcy,Matt Seifert",RadioFree.Net,"Single-player,Multi-player,PvP,Online PvP,Cros...","Casual,Indie,Strategy","Strategy,Card Battler,PvP,Casual,Card Game,2D,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,True,True,True,17,7,1,1,1,6,2023,17,1.833333
68075,September 7th,2023-01-06,0 - 20000,28,0,6.99,0,September 7th is a psychological horror in the...,"'English', 'Russian', 'Portuguese - Portugal'",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://www.emikagames.com/,https://www.emikagames.com/,hawkeys1991@gmail.com,True,False,False,0,NaN,0,51,2,NaN,13,0,The game contains images of scenes of violence...,0,0,0,0,EMIKA_GAMES,EMIKA_GAMES,"Single-player,Steam Achievements,Partial Contr...","Indie,Simulation","Exploration,3D Platformer,Walking Simulator,3D...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,True,True,True,20,14,1,3,1,6,2023,54,17.000000
68078,DARK NIGHT WITH FLOPPA,2023-01-06,0 - 20000,0,0,0.74,0,DARK NIGHT WITH FLOPPA is an emotional horror ...,'English',[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,astra_community@mail.ru,True,False,False,0,NaN,0,31,3,NaN,14,0,NaN,0,0,0,0,Performance Artist,Performance Artist,Single-player,"Action,Adventure,Indie,Simulation","Memes,Funny,Comedy,Horror,Dark,3D,Sandbox,Emot...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,True,True,False,20,15,1,1,1,6,2023,35,7.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85013,Froguelike,2024-01-03,0 - 20000,15,0,2.99,0,Face off against the swarms of bugs invading y...,'English',[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,N

In [ ]:
filter_dict = {
    "metacritic_score": 0,
    "total_reviews": 10,
    "year": 2023,
    "genres": "Indie",
}

In [11]:
X_transformed = steam_df.copy()
X_transformed = X_transformed[
    X_transformed[list(filter_dict.keys())[3]]
    .str.contains(list(filter_dict.values())[3])
    .fillna(False)
]
# X_transformed = X_transformed[X_transformed[list(filter_dict.keys())[0]] == list(filter_dict.values())[0]]
# X_transformed = X_transformed[X_transformed[list(filter_dict.keys())[1]] > list(filter_dict.values())[1]]
# X_transformed = X_transformed[X_transformed[list(self.filter_dict.keys())[2]] >= list(self.filter_dict.values())[2]]